In [1]:
import io
import requests
from PIL import Image
from torchvision import models, transforms
from torch.autograd import Variable
import torchvision.datasets as datasets
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from random import shuffle
import torch.nn.functional as F
import torch.nn as nn

In [2]:
transform = transforms.Compose([
    transforms.Scale(130),
    transforms.RandomCrop(128),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
])

In [3]:
traindir = "/Users/Dylan/Desktop/dogcat/train/"
validdir = "/Users/Dylan/Desktop/dogcat/smallvalidate/"

In [4]:
train = datasets.ImageFolder(traindir, transform)
valid = datasets.ImageFolder(validdir, transform)

In [5]:
def convertImage(data):
    return [data[0][:1], data[1]]

valid = [convertImage(i) for i in valid]
train = [convertImage(i) for i in train]

In [6]:
train[0]

[
 ( 0 ,.,.) = 
   0.5608  0.5765  0.5765  ...   0.7725  0.7725  0.7804
   0.5608  0.5765  0.5765  ...   0.7725  0.7725  0.7804
   0.5765  0.5922  0.5922  ...   0.7725  0.7725  0.7804
            ...             ⋱             ...          
  -0.1922 -0.2784 -0.3569  ...  -0.4824 -0.5373 -0.6000
  -0.1765 -0.2627 -0.3412  ...  -0.6784 -0.7098 -0.7255
  -0.1765 -0.2471 -0.3255  ...  -0.8588 -0.8588 -0.8510
 [torch.FloatTensor of size 1x128x128], 0]

In [42]:
EPOCH = 50
BATCH_SIZE = 50
LR = 1e-4

In [8]:
train_loader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)

In [9]:
valid_y = [i[1] for i in valid]

In [10]:
valid_y = torch.from_numpy(np.array(valid_y))

In [11]:
valid_x = [i[0] for i in valid]

In [12]:
valid_x = torch.stack(valid_x)

In [13]:
valid_x.size()

torch.Size([40, 1, 128, 128])

In [14]:
valid_y.size()

torch.Size([40])

In [15]:
validation_x = Variable(valid_x)

In [43]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 3 * 3, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 2),
            nn.Softmax(),
        )
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x, None

In [44]:
cnn = CNN()
print(cnn)

CNN (
  (features): Sequential (
    (0): Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU (inplace)
    (2): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU (inplace)
    (5): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU (inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU (inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU (inplace)
    (12): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
  )
  (classifier): Sequential (
    (0): Dropout (p = 0.5)
    (1): Linear (2304 -> 4096)
    (2): ReLU (inplace)
    (3): Dropout (p = 0.5)
    (4): Linear (4096 -> 4096)
    (5): ReLU (inplace)
    (6): Linear (4096 -> 2)
    (7): Softmax ()
  )
)


In [45]:
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss()

In [46]:
for epoch in range(EPOCH):
    for step, (x, y) in enumerate(train_loader):
        b_x = Variable(x)
        b_y = Variable(y)
        output = cnn(b_x)[0]               # cnn output
        loss = loss_func(output, b_y)   # cross entropy loss
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()  
        if step % 20 == 0:
            test_output, last_layer = cnn(validation_x)
            pred_y = torch.max(test_output, 1)[1].data.squeeze()
            accuracy = sum(pred_y == valid_y) / float(valid_y.size(0))
            print('Epoch: ', epoch, step,'| train loss: %.4f' % loss.data[0], '| test accuracy: %.2f' % accuracy)
#         print(b_y)
#         break
#     break

Epoch:  0 0 | train loss: 0.6927 | test accuracy: 0.50
Epoch:  0 20 | train loss: 0.6944 | test accuracy: 0.50
Epoch:  0 40 | train loss: 0.6953 | test accuracy: 0.50
Epoch:  0 60 | train loss: 0.6918 | test accuracy: 0.57
Epoch:  0 80 | train loss: 0.6953 | test accuracy: 0.50
Epoch:  0 100 | train loss: 0.6821 | test accuracy: 0.62
Epoch:  0 120 | train loss: 0.6739 | test accuracy: 0.62
Epoch:  0 140 | train loss: 0.6346 | test accuracy: 0.65
Epoch:  0 160 | train loss: 0.6361 | test accuracy: 0.50
Epoch:  0 180 | train loss: 0.6869 | test accuracy: 0.65
Epoch:  0 200 | train loss: 0.6088 | test accuracy: 0.53
Epoch:  0 220 | train loss: 0.6173 | test accuracy: 0.57
Epoch:  0 240 | train loss: 0.6293 | test accuracy: 0.57
Epoch:  0 260 | train loss: 0.6061 | test accuracy: 0.55
Epoch:  0 280 | train loss: 0.6645 | test accuracy: 0.55
Epoch:  0 300 | train loss: 0.5532 | test accuracy: 0.60
Epoch:  0 320 | train loss: 0.6224 | test accuracy: 0.62
Epoch:  0 340 | train loss: 0.5677 | 

KeyboardInterrupt: 

In [85]:
t1 = train[4][0]

In [82]:
t1 = torch.stack([t1])

In [83]:
r = cnn(Variable(t1))

In [84]:
r[0]

Variable containing:
 0.5000  0.5000
[torch.FloatTensor of size 1x2]

In [141]:
train[0][0]


( 0 ,.,.) = 
  0.6078  0.6157  0.6235  ...   0.7725  0.7647  0.7490
  0.6078  0.6157  0.6157  ...   0.7725  0.7647  0.7569
  0.6078  0.6157  0.6078  ...   0.7804  0.7725  0.7725
           ...             ⋱             ...          
 -0.6471 -0.6235 -0.6000  ...  -0.9765 -0.9765 -0.9686
 -0.6471 -0.6235 -0.6235  ...  -0.9765 -0.9765 -0.9843
 -0.6706 -0.6549 -0.6471  ...  -0.9843 -0.9843 -0.9922
[torch.FloatTensor of size 1x128x128]